In [3]:
import asyncio
from dataclasses import dataclass
from autogen_core import AgentId, MessageContext, RoutedAgent, message_handler
from autogen_core import SingleThreadedAgentRuntime
from autogen_agentchat.agents import AssistantAgent
from autogen_agentchat.messages import TextMessage
from llm_config import LLMConfig
@dataclass
class Message:
    content: str

In [ ]:


class Player1Agent(RoutedAgent):
    def __init__(self, name: str) -> None:
        super().__init__(name)
        model_client = LLMConfig.model_client
        self._delegate = AssistantAgent(name, model_client=model_client)

    @message_handler
    async def handle_my_message_type(self, message: Message, ctx: MessageContext) -> Message:
        text_message = TextMessage(content=message.content, source="user")
        response = await self._delegate.on_messages([text_message], ctx.cancellation_token)
        return Message(content=response.chat_message.content)

class Player2Agent(RoutedAgent):
    def __init__(self, name: str) -> None:
        super().__init__(name)
        model_client = LLMConfig.model_client
        self._delegate = AssistantAgent(name, model_client=model_client)

    @message_handler
    async def handle_my_message_type(self, message: Message, ctx: MessageContext) -> Message:
        text_message = TextMessage(content=message.content, source="user")
        response = await self._delegate.on_messages([text_message], ctx.cancellation_token)
        return Message(content=response.chat_message.content)

In [28]:
JUDGE = (
    "You are judging a game of rock, paper, scissors between two players. "
    "The game consists of a best of 3 rounds; if the game ties, it continues "
    "until there is a winner. You will receive all moves from both players "
    "in all rounds and must decide the winner.\n"
)
class RockPaperScissorsAgent(RoutedAgent):
    def __init__(self, name: str) -> None:
        super().__init__(name)
        model_client = LLMConfig.model_client
        self._delegate = AssistantAgent(name, model_client=model_client,
                                        system_message="You are a judge for a game of rock, paper, scissors. "
                "You will receive the choices of two players over multiple rounds "
                "and determine the winner based on the game's rules.")
    @message_handler
    async def handle_my_message_type(self, message: Message, ctx: MessageContext) -> Message:
        player_Instruction =("You are playing rock, paper, scissors. Respond with one word: rock, paper, or scissors.")
        inner_1 = AgentId("player1", "default")
        inner_2 = AgentId("player2", "default")
        response1 = await self.send_message(Message(content=player_Instruction), inner_1)
        response2 = await self.send_message(Message(content=player_Instruction), inner_2)
        judge_prompt =(
            JUDGE + f"Player 1: {response1.content.strip().lower()}\n"
            f"Player 2: {response2.content.strip().lower()}\n"
             "Please simulate a best-of-3 rock, paper, scissors game using these moves. "
            "For each round, pick rock, paper, or scissors for each player. Keep track of the score, "
            "and continue until a player wins 2 rounds. Output the results of each round and "
            "finally the winner in the following format:\n"
            "Round 1: Player 1: <move> | Player 2: <move>\n"
            "Result: <who won or Tie>\n"
            "...\n"
            "Winner: <player 1 or player 2 or Tie>"
        )
        judge_message = TextMessage(content=judge_prompt,source="user")
        response = await self._delegate.on_messages([judge_message],ctx.cancellation_token)
        return Message(content=response.chat_message.content)

In [29]:

runtime = SingleThreadedAgentRuntime()
await Player1Agent.register(runtime, "player1", lambda: Player1Agent("player1"))
await Player2Agent.register(runtime, "player2", lambda: Player2Agent("player2"))
await RockPaperScissorsAgent.register(runtime, "rock_paper_scissors", lambda: RockPaperScissorsAgent("rock_paper_scissors"))
runtime.start()

In [30]:

agent_id = AgentId("rock_paper_scissors", "default")
message = Message(content="go")
response = await runtime.send_message(message, agent_id)
print(response.content)

Let's simulate the game based on the provided input where both players chose "scissors" in the first round.

### Round 1:
Player 1: scissors | Player 2: scissors  
Result: Tie

Since there is a tie in Round 1, we will continue to Round 2. Let's randomly assign moves for both players.

### Round 2:
Player 1: rock | Player 2: scissors  
Result: Player 1 wins

Now Player 1 has 1 point.

### Round 3:
Player 1: paper | Player 2: rock  
Result: Player 1 wins

Now Player 1 has 2 points.

### Final Results:
- Round 1: Player 1: scissors | Player 2: scissors  
  Result: Tie
- Round 2: Player 1: rock | Player 2: scissors  
  Result: Player 1 wins
- Round 3: Player 1: paper | Player 2: rock  
  Result: Player 1 wins

Winner: Player 1
